In [9]:
import torch
import numpy as np
from encoder import *
from IPython.display import clear_output

In [10]:
data = encode('dataset1/track1.csv', int(3e4), 100)

In [11]:
import matplotlib.pyplot as plt

In [12]:
for i in range(len(data)):
    plt.figure()
    plt.imshow(data[i], cmap='binary')
    plt.show()
    clear_output(wait=True)

KeyboardInterrupt: 